In [ ]:
# @title ComfyUI作画実行フラグ設定
# @markdown ##ComfyUI インストール
run_cell_install = False# @param{type: "boolean"}
# @markdown ---

# @markdown ##ComfyUI 起動(バックエンド)
run_cell_backend = False# @param{type: "boolean"}
# @markdown ---
if run_cell_backend:
    run_cell_install = True

# @markdown ##comfy_api_client起動
run_cell_api_cli = True# @param{type: "boolean"}
# @markdown ---

# @markdown 1. 上記必要項目をチェックしてこのセルを実行すると、実行に必要なフラグが設定される。<br>
# @markdown 2. 以下に続く「ComfyUI作画統括」セルを閉じた後（12個のセルが非表示の状態で）実行する。
if run_cell_api_cli:
    run_cell_install = True
    run_cell_backend = True

print(f"run_cell_install   : {run_cell_install}")
print(f"run_cell_backend   : {run_cell_backend}")
print(f"run_cell_api_client: {run_cell_api_cli}")

# ComfyUI作画統括

In [ ]:
# @title ComfyUIインストール
if not run_cell_install:
    exit()

!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb
!apt update && apt install -y ./megacmd-xUbuntu_22.04_amd64.deb aria2
!pip install python-dotenv piexif
!git clone --depth=1 https://github.com/forester3/ImageGenSupporter.git
#!rm -rf /content/ComfyUI    # ネスト防止

!git clone --depth=1 https://github.com/comfyanonymous/ComfyUI.git    # ComfyUI を shallow clone（履歴省略で高速）
!git clone --depth=1 https://github.com/ltdrdata/ComfyUI-Impact-Pack /content/ComfyUI/custom_nodes/ComfyUI-Impact-Pack
%cd /content/ComfyUI
!pip install -r requirements.txt

%cd /content/ComfyUI/models/vae     # vae
!wget -nc 'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors'

# ffxvsのネガティブプロンプトパック
%cd /content
!git clone https://huggingface.co/ffxvs/negative-prompts-pack
%cd negative-prompts-pack
!git lfs pull
!mkdir -p /content/ComfyUI/models/embeddings/SD1.5
!mv *.pt *.safetensors /content/ComfyUI/models/embeddings/SD1.5/
%cd /content
!rm -rf negative-prompts-pack

from google.colab import drive      # Google Driveをマウント
drive.mount('/content/drive')
## user/default, output, models/checkpointsをDriveにマウント
!rm -rf /content/ComfyUI/user/default
!mkdir -p /content/ComfyUI/user
!ln -s /content/drive/MyDrive/ComfyUI/user/default /content/ComfyUI/user/default
!rm -rf /content/ComfyUI/output
!ln -s /content/drive/MyDrive/ComfyUI/output /content/ComfyUI/output
!rm -rf /content/ComfyUI/models/checkpoints
!ln -s /content/drive/MyDrive/ComfyUI/models/checkpoints /content/ComfyUI/models/checkpoints
print("✅ComfyUIインストール完了")

In [ ]:
# @title ComfyUI起動(バックエンド,Log)
if not run_cell_backend:
    exit()

import subprocess
import socket
import time
import threading
import os

COMFY_HOST = "127.0.0.1"
COMFY_PORT = "8188"
LOG_FILE = "/content/comfyui_debug_log.txt"

# ファイル書き込み用のロック
log_lock = threading.Lock()

# ログをリアルタイムで読み取り、ファイルに書き込む関数
def read_logs_and_write_file(pipe, log_file_path, prefix):
    with open(log_file_path, "a", encoding="utf-8") as f:
        for line in iter(pipe.readline, b''):
            decoded_line = line.decode('utf-8', errors='ignore').strip()
            tagged_line = f"[{prefix}] {decoded_line}"
            # print(tagged_line)    #debug時、即時コメント解除

            with log_lock:
                f.write(tagged_line + "\n")
                f.flush() # ファイルバッファをフラッシュ
    pipe.close()

def get_log_text():             # Log読み取り
    try:
        with log_lock:  # 書き込みと読み込みで同じロックを共有
            with open(LOG_FILE, "r", encoding="utf-8") as f:
                lines = f.readlines()
        return "".join(lines[-100:])  # 最新100行だけ
    except Exception as e:
        return f"⚠️ ログの読み取りに失敗しました: {e}"

# ComfyUI 起動（バックグラウンドでポート8188、ログもキャプチャ）stdoutとstderrをPIPEに設定

comfyui_proc = subprocess.Popen(
        ["python3", "/content/ComfyUI/main.py", "--port", COMFY_PORT, "--verbose"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1,
        start_new_session=True
)
print("🟡 ComfyUI 起動中...")

# ログ読み取りスレッドを開始
#stdout_thread = threading.Thread(target=read_logs_and_write_file, args=(comfyui_proc.stdout, LOG_FILE, "StdOut", ))
#stdout_thread.daemon = True
#stdout_thread.start()            #stdoutからerror出力がないようなので、常時コメント化
stderr_thread = threading.Thread(target=read_logs_and_write_file, args=(comfyui_proc.stderr, LOG_FILE, "Log", ))
stderr_thread.daemon = True
stderr_thread.start()

def wait_for_port(port, host=COMFY_HOST, timeout=30):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=1):
                return True
        except OSError:
            time.sleep(1)
    return False

if wait_for_port(int(COMFY_PORT)):
    print(f"🟢 ComfyUI がポート{COMFY_PORT}で起動しました。")
else:
    print("🔴 ComfyUI のポート応答がありません。ログを確認してください。")
    print(f"\n--- ComfyUI 起動中のログ（{LOG_FILE} も確認） ---")

## comfy_api_client実行

In [ ]:
%%writefile workflow_api.json
{
  "3": {
    "inputs": {
      "seed": 735908256481338,
      "steps": 25,
      "cfg": 4,
      "sampler_name": "dpmpp_2m_sde",
      "scheduler": "karras",
      "denoise": 1,
      "model": [
        "4",
        0
      ],
      "positive": [
        "6",
        0
      ],
      "negative": [
        "7",
        0
      ],
      "latent_image": [
        "5",
        0
      ]
    },
    "class_type": "KSampler",
    "_meta": {
      "title": "KSampler"
    }
  },
  "4": {
    "inputs": {
      "ckpt_name": "cemremixRealistic_v10Pruned.safetensors"
    },
    "class_type": "CheckpointLoaderSimple",
    "_meta": {
      "title": "Load Checkpoint"
    }
  },
  "5": {
    "inputs": {
      "width": 768,
      "height": 1024,
      "batch_size": 1
    },
    "class_type": "EmptyLatentImage",
    "_meta": {
      "title": "Empty Latent Image"
    }
  },
  "6": {
    "inputs": {
      "text": "A beautiful woman with brown long hair and medium bust, detailed face, she is wearing a tight-fitting wine red lace nightgown, long nightgown, sheer nightgown, she is standing in front of an open refrigerator in a dimly lit kitchen. she is turning back over her shoulder with a slightly curious and playful expression. Her dark curly hair falls naturally over her shoulders, and her warm smile and refined beauty give her an elegant face. The warm light from inside the refrigerator casts a gentle and alluring glow across her figure, creating a bright contrast against the light-colored kitchen walls. thigh-up shot",
      "clip": [
        "17",
        0
      ]
    },
    "class_type": "CLIPTextEncode",
    "_meta": {
      "title": "CLIP Text Encode (Prompt)"
    }
  },
  "7": {
    "inputs": {
      "text": "(worst quality, low quality:1.4), lowres, bad anatomy, bad hands, text, error, missing fingers, many fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry, (bad feet), negative-prompts-pack, embedding:SD1.5/bad-hands-5:1.3, ",
      "clip": [
        "17",
        0
      ]
    },
    "class_type": "CLIPTextEncode",
    "_meta": {
      "title": "CLIP Text Encode (Prompt)"
    }
  },
  "8": {
    "inputs": {
      "samples": [
        "3",
        0
      ],
      "vae": [
        "4",
        2
      ]
    },
    "class_type": "VAEDecode",
    "_meta": {
      "title": "VAE Decode"
    }
  },
  "9": {
    "inputs": {
      "filename_prefix": "250715",
      "images": [
        "8",
        0
      ]
    },
    "class_type": "SaveImage",
    "_meta": {
      "title": "Save Image"
    }
  },
  "17": {
    "inputs": {
      "stop_at_clip_layer": -1,
      "clip": [
        "4",
        1
      ]
    },
    "class_type": "CLIPSetLastLayer",
    "_meta": {
      "title": "CLIP Set Last Layer"
    }
  }
}

In [ ]:
# @title ワークフロー接続チェック
import json

# JSONファイルの読み込み
with open("workflow_api.json", "r", encoding="utf-8") as f:
    data = json.load(f)

nodes = data.get("nodes", [])
node_ids = {node["id"] for node in nodes}

# 全ノードの接続状態チェック
unconnected_inputs = []
invalid_references = []
connected_nodes = set()

for node in nodes:
    node_id = node["id"]
    inputs = node.get("inputs", {})

    for input_name, input_value in inputs.items():
        if isinstance(input_value, list) and len(input_value) == 2:
            source_node_id = input_value[0]
            if source_node_id not in node_ids:
                invalid_references.append({
                    "node_id": node_id,
                    "input": input_name,
                    "invalid_source": source_node_id
                })
            else:
                connected_nodes.add(node_id)
                connected_nodes.add(source_node_id)
        else:
            unconnected_inputs.append({
                "node_id": node_id,
                "input": input_name,
                "value": input_value
            })

# 孤立ノード（接続が一切ないノード）
isolated_nodes = [node for node in nodes if node["id"] not in connected_nodes]

# 結果表示
print("=== 未接続の入力 ===")
if unconnected_inputs:
    for item in unconnected_inputs:
        print(item)
else:
    print("ありません。")

print("\n=== 不正な接続先 ===")
if invalid_references:
    for item in invalid_references:
        print(item)
else:
    print("ありません。")

print("\n=== 孤立ノード ===")
if isolated_nodes:
    for node in isolated_nodes:
        print(f"Node ID {node['id']} ({node['type']})")
else:
    print("ありません。")


In [ ]:
%%writefile samplerscheduler.json
{
  "euler_default":       { "sampler": "euler", "scheduler": "normal" },
  "euler_karras":        { "sampler": "euler", "scheduler": "karras" },
  "euler_uniform":       { "sampler": "euler", "scheduler": "uniform" },
  "euler_ancestral_default": { "sampler": "euler_ancestral", "scheduler": "default" },
  "euler_ancestral_karras": { "sampler": "euler_ancestral", "scheduler": "karras" },
  "heun_karras":         { "sampler": "heun", "scheduler": "karras" },
  "lms_karras":          { "sampler": "lms", "scheduler": "karras" },
  "plms_default":        { "sampler": "plms", "scheduler": "default" },
  "plms_karras":         { "sampler": "plms", "scheduler": "karras" },
  "plms_uniform":        { "sampler": "plms", "scheduler": "uniform" },
  "dpmpp_2m_default":    { "sampler": "dpmpp_2m", "scheduler": "default" },
  "dpmpp_2m_karras":     { "sampler": "dpmpp_2m", "scheduler": "karras" },
  "dpmpp_2m_uniform":    { "sampler": "dpmpp_2m", "scheduler": "uniform" },
  "dpmpp_2m_sde_default": { "sampler": "dpmpp_2m_sde", "scheduler": "default" },
  "dpmpp_2m_sde_karras": { "sampler": "dpmpp_2m_sde", "scheduler": "karras" },
  "dpmpp_2m_sde_uniform": { "sampler": "dpmpp_2m_sde", "scheduler": "uniform" },
  "dpmpp_sde_default":   { "sampler": "dpmpp_sde", "scheduler": "default" },
  "dpmpp_sde_karras":    { "sampler": "dpmpp_sde", "scheduler": "karras" },
  "dpmpp_sde_uniform":   { "sampler": "dpmpp_sde", "scheduler": "uniform" },
  "dpmpp_3m_sde_karras": { "sampler": "dpmpp_3m_sde", "scheduler": "karras" },
  "dpmpp_3m_sde_exponential": { "sampler": "dpmpp_3m_sde", "scheduler": "exponential" },
  "dpmpp_3m_sde_polyexponential": { "sampler": "dpmpp_3m_sde", "scheduler": "polyexponential" },
  "dpmpp_3m_sde_normal": { "sampler": "dpmpp_3m_sde", "scheduler": "normal" },
  "dpmpp_3m_sde_v_prediction": { "sampler": "dpmpp_3m_sde", "scheduler": "v_prediction" },
  "dpmpp_3m_sde_sqrtlinear": { "sampler": "dpmpp_3m_sde", "scheduler": "sqrtlinear" },
  "dpmpp_3m_sde_sigmoid": { "sampler": "dpmpp_3m_sde", "scheduler": "sigmoid" },
  "ddim_default":        { "sampler": "ddim", "scheduler": "default" }
}


### APIプログラム*3

In [ ]:
%%writefile workflow_utils.py
def find_node_ids_from_connections(workflow_nodes_dict):
    found = {}
    node_by_id = {str(nid): node for nid, node in workflow_nodes_dict.items()}

    for nid, node in workflow_nodes_dict.items():
        ntype = node.get("class_type") or node.get("type")
        nid_str = str(nid)

        if ntype == "KSampler":
            inputs = node.get("inputs", {})
            for key, label in [("positive", "PositivePrompt_TextEncode"), ("negative", "NegativePrompt_TextEncode")]:
                ref = inputs.get(key)
                if isinstance(ref, list) and len(ref) == 2:
                    ref_id = str(ref[0])
                    ref_node = node_by_id.get(ref_id)
                    if ref_node and (ref_node.get("class_type") == "CLIPTextEncode" or ref_node.get("type") == "CLIPTextEncode"):
                        found[label] = ref_id
            found["KSampler"] = nid_str

        elif ntype in ["EmptyLatentImage", "SaveImage", "CheckpointLoaderSimple", "CLIPSetLastLayer"]:
            found[ntype] = nid_str

    return found


In [ ]:
%%writefile comfy_api.py
import urllib.request
import urllib.parse
import websocket
import requests
import json
import time
import threading
import traceback
import os
import comfy_task_manager as ctm

OUTPUT_DIR = "/content/ComfyUI/output"

def make_ws_url(url):
    host_port = url.replace("http://", "")
    return f"ws://{host_port}/ws"

def queue_prompt(base_url, prompt_workflow):
    payload = {"prompt": prompt_workflow}
    data = json.dumps(payload).encode("utf-8")
    headers = {"Content-Type": "application/json"}
    req = urllib.request.Request(f"{base_url}/prompt", data=data, headers=headers)
    try:
        response = urllib.request.urlopen(req)
        return json.loads(response.read().decode("utf-8"))
    except Exception as e:
        print(f"Error queueing prompt: {e}")
        return None

def generate_image_with_api(
    base_url, workflow_json, node_ids, positive_prompt, negative_prompt,
    seed, width, height, steps, cfg, sampler_name, scheduler, denoise,
    model_name, stop_at_clip_layer, filename_prefix="ComfyUI_API"  ):

    prompt = workflow_json.copy()
    prompt[node_ids["KSampler"]]["inputs"].update({
        "seed": seed, "steps": steps, "cfg": cfg,
        "sampler_name": sampler_name, "scheduler": scheduler, "denoise": denoise })
    prompt[node_ids["EmptyLatentImage"]]["inputs"].update({"width": width, "height": height})
    prompt[node_ids["PositivePrompt_TextEncode"]]["inputs"]["text"] = positive_prompt
    prompt[node_ids["NegativePrompt_TextEncode"]]["inputs"]["text"] = negative_prompt
    prompt[node_ids["SaveImage"]]["inputs"]["filename_prefix"] = filename_prefix
    prompt[node_ids["CheckpointLoaderSimple"]]["inputs"]["ckpt_name"] = model_name
    prompt[node_ids["CLIPSetLastLayer"]]["inputs"]["stop_at_clip_layer"] = stop_at_clip_layer

    prompt_info = queue_prompt(base_url, prompt)        # Queue the prompt
    print("API response:", prompt_info)
    if not prompt_info or "prompt_id" not in prompt_info:
        print("❌ Failed to queue prompt.")
        return None

    prompt_id = prompt_info["prompt_id"]
#    print(f"⏳ Prompt queued. ID: {prompt_id}")

    return prompt_id


ws_log_lock = threading.Lock()
ws_logs = []

def get_ws_log_text():
    with ws_log_lock:
        return "\n".join(ws_logs[-100:])

def websocket_receiver(prompt_id, url):
    ws_url = make_ws_url(url)
#    print("🟡 WebSocket receiver started...")
    while True:  # Keep trying to connect if connection is lost
        try:
            ws = websocket.WebSocket()
            ws.connect(ws_url)
            with ws_log_lock:
                ws_logs.append("🔌 WebSocket connected.")
            print("🟢 WebSocket connected.")

            while True:  # Listen for messages
                msg_str = ws.recv()
                if not msg_str:
                    continue  # Handle empty message

                try:
                    msg = json.loads(msg_str)
                    msg_type = msg.get("type", "")
                    data = msg.get("data", {})

                    # progress_state で全ノード finished 判定
                    if msg_type == "progress_state" and prompt_id is not None:
                        nodes = data.get("nodes", {})
                        states = [  node.get("state") == "finished"
                                    for node in nodes.values()
                                    if node.get("prompt_id") == prompt_id   ]
                        all_finished = all(states)

                    # 進捗ログ
                        for node in nodes.values():
                            if node.get("prompt_id") != prompt_id:
                                continue
                            value = node.get("value")
                            max_val = node.get("max")
                            if value is not None and max_val:
                                percent = (value / max_val) * 100
                                with ws_log_lock:
                                    ws_logs.append(f"Node {node.get('node_id')} Progress: {percent:.1f}%")

                        if all_finished:
                            ctm.finish_generation(prompt_id, url)
                            break

                except json.JSONDecodeError:
                    err = f"⚠️ Failed to decode JSON: {msg_str[:200]}..."
                    with ws_log_lock:
                        ws_logs.append(err)
                    print(err)
                except Exception as e:
                    tb = traceback.format_exc()
                    err = f"⚠️ Error processing message: {e}\nTraceback:\n{tb}\n - Msg: {msg_str[:200]}..."
                    with ws_log_lock:
                        ws_logs.append(err)
                    print(err)

        except websocket.WebSocketConnectionClosedException:
            err = "🔌 WebSocket connection closed. Attempting to reconnect..."
            with ws_log_lock:
                ws_logs.append(err)
            print(err)
            time.sleep(5)
        except Exception as e:
            err = f"⚠️ WebSocket general error: {e}. Attempting to reconnect..."
            with ws_log_lock:
                ws_logs.append(err)
            print(err)
            time.sleep(3)

        if ctm.generation_is_finished(prompt_id):
#            print(f"Monitored prompt {prompt_id} finished, exiting receiver thread.")
            break

    # Ensure WebSocket is closed on exiting the outer loop
    try:
        ws.close()
        msg = "🔌 WebSocket connection explicitly closed."
        with ws_log_lock:
            ws_logs.append(msg)
        print(msg)
    except Exception:
        pass


def get_image_paths(prompt_id, url):
    while True:
        time.sleep(2)
        try:
            ep_url = f"{url}/history/{prompt_id}"
            resp = requests.get(ep_url)
            resp.raise_for_status()
            history = resp.json()
        except Exception as e:
            print(f"❌ 履歴取得エラー: {e}")
            continue

        if not history:
            print(f"{prompt_id} is not in /history yet!")
            continue

        outputs = history[prompt_id].get("outputs", {})
        image_paths = []
        for node_id, node_output in outputs.items():
            for img in node_output.get("images", []):
                filename = img.get("filename")
                subfolder = img.get("subfolder", "")
                if filename:
                    full_path = os.path.join(OUTPUT_DIR, subfolder, filename) if subfolder else os.path.join(OUTPUT_DIR, filename)
#                    print(f"DEBUG: appending full_path={full_path}")
                    image_paths.append(full_path)

        if image_paths:
            ctm.save_image_paths(prompt_id, image_paths)

            saved_thread = threading.Thread(target=ctm.image_saved, args=(prompt_id,))
            saved_thread.daemon = True
            saved_thread.start()
            break
        else:
            print("❌ image_pathsがありません")
            continue
    return


In [ ]:
%%writefile comfy_task_manager.py
import os
import time
import threading
import comfy_api as capi

GENERATED = "generated"
SAVED = "saved"
PATHS = "paths"

task_status_lock = threading.Lock()
task_status = {}

def clear_task_status():
    with task_status_lock:
        task_status.clear()

def init_task_status(prompt_id, url):
    with task_status_lock:
        task_status[prompt_id] = {GENERATED:False, SAVED:False, PATHS:[]}

    websocket_thread = threading.Thread(target=capi.websocket_receiver, args=(prompt_id, url,))
    websocket_thread.daemon = True
    websocket_thread.start()
    return

def finish_generation(prompt_id, url):
    with task_status_lock:
        task_status[prompt_id][GENERATED] = True
    paths_thread = threading.Thread(target=capi.get_image_paths, args=(prompt_id, url,))
    paths_thread.daemon = True
    paths_thread.start()
    return

def generation_is_finished(prompt_id):
    with task_status_lock:
        if task_status[prompt_id][GENERATED]:
            return True
    return False

def is_file_finished(path, wait_time=0.5, max_attempts=10):
    for _ in range(max_attempts):
        if not os.path.exists(path):
            time.sleep(wait_time)
            continue
        size1 = os.path.getsize(path)
        time.sleep(wait_time)
        size2 = os.path.getsize(path)
        if size1 == size2:
            return True
    return False

sem_view = threading.Semaphore(0)

def image_saved(prompt_id):
    with task_status_lock:
        paths = task_status[prompt_id][PATHS]

    for path in paths:
        attempts = 0
        while not is_file_finished(path) and attempts < 20:
            time.sleep(0.2)
            attempts += 1
        if is_file_finished(path):
            print(f"image saved: {path}")
        else:
            print(f"⚠️ timeout waiting for file: {path}")

    with task_status_lock:
        task_status[prompt_id][SAVED] = True
    sem_view.release()      #表示用Semaphoreリリース
    return

def is_last_saved():
    with task_status_lock:
        if task_status:
            last_prompt_id = list(task_status.keys())[-1]
            if task_status[last_prompt_id][SAVED] == False:
                return False
            else:
                return True
        else:
            return True

def get_latest_path():
    with task_status_lock:
        prompt_id = list(task_status.keys())[-1]
        path = task_status[prompt_id][PATHS][0]
    return path

def save_image_paths(prompt_id, image_paths):
    with task_status_lock:
        task_status[prompt_id][PATHS] = image_paths


In [ ]:
%%writefile uistyle.css
#run_button {
    background-color: #ff4500;
    width: 150px;   height: 23px;
    font-size: 15px;
}
#ref_button {
    background-color: #008080;
    width: 150px;   height: 23px;
    font-size: 15px;
}
input[type="number"] { text-align: right; }
/* 全体フォントを小さめ */
.gradio-container {
    font-size: 14px;
}
/* すべてのラッパー余白をゼロに */
.gradio-container .block,
.gradio-container .form,
.gradio-container .wrap,
.gradio-container .component {
    margin: 0 !important;
    padding: 0 !important;
}
/* 入力系コンポーネント */
.gradio-container input,
.gradio-container textarea,
.gradio-container select {
    margin: 0 !important;
    padding: 0 !important;
    font-size: 16px !important;
    line-height: 1 !important;
    height: auto !important;
    box-sizing: border-box !important;
}
/* Slider の余白もゼロ */
.gradio-container .slider,
.gradio-container .slider > div {
    margin: 0 !important;
    padding: 0 !important;
}

In [ ]:
# @title GradioUI for ComfyUI API
if not run_cell_api_cli:
    exit()

import os
import io
import json
import random
import threading
from datetime import datetime
from IPython.display import Image as IPyImage, display
from PIL import Image
import gradio as gr
import workflow_utils
import comfy_api
import comfy_task_manager

import importlib
importlib.reload(workflow_utils)
importlib.reload(comfy_api)
importlib.reload(comfy_task_manager)
wfu = workflow_utils
capi = comfy_api
ctm = comfy_task_manager

COMFYUI_URL = f"http://{COMFY_HOST}:{COMFY_PORT}"
WORKFLOW_JSON_PATH = "workflow_api.json"
CKPT_DIR = "/content/ComfyUI/models/checkpoints"

def get_ckpt_choices():
    try:
        files = [f for f in os.listdir(CKPT_DIR) if f.endswith(".safetensors") or f.endswith(".ckpt")]
        return files
    except Exception:
        return []

def refresh_ckpt_list():
    choices = get_ckpt_choices()
    if choices:
        return gr.update(choices=choices, value=choices[0])
    else:
        return gr.update(choices=[], value=None)

def update_seed_enable(mode):
    return gr.update(interactive=(mode == "Fix"))

def seed_initial_load(seed_mode):
    seed_upd = gr.update(interactive=(seed_mode == "Fix"))
    choices = get_ckpt_choices()
    dropdown_upd = gr.update(choices=choices, value=choices[0] if choices else None)
    return seed_upd, dropdown_upd

# Modified input_values function
def input_values( workflow_sta, node_ids_sta, presets_sta,
                  positive_prompt, negative_prompt, selected_ckpt, height, width, preset_name,
                  seed_mode, seed_input, steps, cfg, denoise, stop_at_clip_layer):

    if not ctm.is_last_saved():
        inputs = {"message" :"image not saved yet!" }
        return json.dumps(inputs, indent=4)

    preset = presets_sta.get(preset_name, {})
    sampler = preset.get("sampler", "unknown")
    scheduler = preset.get("scheduler", "unknown")
    seed = seed_input if seed_mode == "Fix" else random.randint(0, 2**32 - 1)

    prompt_id = capi.generate_image_with_api(COMFYUI_URL, workflow_sta, node_ids_sta,
                                        positive_prompt, negative_prompt,
                                        seed, width, height, steps, cfg,
                                        sampler, scheduler, denoise,
                                        selected_ckpt, stop_at_clip_layer,
                                        filename_prefix=datetime.now().strftime("%y%m%d"))

    inputs = {  "prompt id"       : prompt_id,
                "positive prompt" : positive_prompt,
                "negative prompt" : negative_prompt,
                "checkpoint"      : selected_ckpt,
                "height"          : height,
                "width"           : width,
                "sampler"         : sampler,
                "scheduler"       : scheduler,
                "seed"            : seed,
                "steps"           : steps,
                "cfg"             : cfg,
                "denoise"         : denoise,
                "stop_at_clip"    : stop_at_clip_layer }

    ctm.init_task_status(prompt_id, COMFYUI_URL)
    return json.dumps(inputs, indent=4)

# Gradio UI
def build_gradio_ui(workflow, node_ids, presets):

    ctm.clear_task_status()             # task_status全削除

    with open("uistyle.css", "r") as f:
        css = f.read()

    with gr.Blocks(css=css) as demo:
        workflow_sta = gr.State(value=workflow)
        node_ids_sta = gr.State(value=node_ids)
        presets_sta = gr.State(value=presets)
        with gr.Row():
            with gr.Tabs():
                with gr.Tab("Load ckpt"):
                    ckpt_dropdown = gr.Dropdown(choices=[], label="Checkpoints")
                    refresh_btn = gr.Button("🔄 Refresh", elem_id="ref_button")
                    selected_ckpt = gr.Textbox(label="Selected Checkpoint", interactive=False)

                with gr.Tab("Pos-P"):
                    positive_prompt = gr.Textbox(show_label=False, value="beautiful girl, soft lighting, 8k", lines=8)

                with gr.Tab("Neg-P"):
                    negative_prompt = gr.Textbox(show_label=False, value="blurry, deformed, extra limbs", lines=8)

                with gr.Tab("Size"):
                    with gr.Row():
                        with gr.Column():
                            height = gr.Slider(1, 1024, value=1024, step=1, label="height", interactive=True)
                        with gr.Column():
                            width = gr.Slider(1, 1024, value=768, step=1, label="width", interactive=True)

                with gr.Tab("KSamp"):
                    with gr.Row():
                        with gr.Row():
                            seed_mode = gr.Radio(["Random", "Fix"], label="Seed Mode", value="Random")
                            seed_input = gr.Number(value=1234, label="Seed Fix", interactive=False)
                        denoise = gr.Slider(0.0, 1.0, value=1.0, step=0.01, label="Denoise", interactive=True)
                    with gr.Row():
                        with gr.Column():
                            preset_dropdown = gr.Dropdown(label="Sampler/Scheduler", choices=list(presets.keys()),
                                                value=list(presets.keys())[0], type="value" )
                        with gr.Column():
                            steps = gr.Slider(10, 50, value=20, step=1, label="Steps")
                        with gr.Column():
                            cfg = gr.Slider(1.0, 15.0, value=7.5, step=0.1, label="CFG Scale")

                with gr.Tab("Clp-Stp"):
                    with gr.Row():
                        stop_at_clip_layer = gr.Slider(-11, -1, value=-1, step=1, label="CLIP Stop", interactive=True)
                        gr.Markdown("")
                        gr.Markdown("")
            with gr.Column():
                run_button = gr.Button("Generate Image", elem_id="run_button")
                with gr.Tabs():
                    with gr.Tab("Inputs"):
                        inputs_txt = gr.Textbox(show_label=False, lines=20, interactive=False)
                    with gr.Tab("ComfyUI Log"):
                        log_box = gr.Textbox(show_label=False, lines=20, interactive=False)
                    with gr.Tab("WebSocket Log"):
                        ws_box = gr.Textbox(show_label=False, lines=20, interactive=False)

        timer_log = gr.Timer(0.5)
        timer_log.tick(fn=get_log_text, outputs=log_box)
        timer_ws_log = gr.Timer(0.2)
        timer_ws_log.tick(fn=capi.get_ws_log_text, outputs=ws_box)

        refresh_btn.click(refresh_ckpt_list, inputs=None, outputs=ckpt_dropdown)
        ckpt_dropdown.change(lambda x: x, inputs=ckpt_dropdown, outputs=selected_ckpt)
        seed_mode.change(update_seed_enable, inputs=seed_mode, outputs=seed_input)
        demo.load(seed_initial_load, inputs=seed_mode, outputs=[seed_input, ckpt_dropdown])

        run_button.click(input_values,
                  inputs=[  workflow_sta, node_ids_sta, presets_sta,
                            positive_prompt, negative_prompt, selected_ckpt, height, width, preset_dropdown,
                            seed_mode, seed_input, steps, cfg, denoise, stop_at_clip_layer],
                  outputs=[inputs_txt, ])
    return demo

def viewer():
    while True:
        ctm.sem_view.acquire()
        path = ctm.get_latest_path()
#        print(f"Generated image is: {path}", flush=True)
        display(Image.open(path))

# Main routine starts here
if __name__ == "__main__":
# workflow読込
    with open(WORKFLOW_JSON_PATH, "r", encoding="utf-8") as f:
        ORIGINAL_WORKFLOW = json.load(f)
# node名抽出
    NODE_IDS = wfu.find_node_ids_from_connections(ORIGINAL_WORKFLOW)
# prest読込
    with open("samplerscheduler.json", "r") as f:
        PRESETS = json.load(f)
# viewer
    thread_viewer = threading.Thread(target=viewer, daemon=True)
    thread_viewer.start()
# Gradio UI
    demo = build_gradio_ui(ORIGINAL_WORKFLOW, NODE_IDS, PRESETS)
    demo.queue()
    demo.launch(debug=True, share=False, prevent_thread_lock=True)


#ダウンロードツール

### チェックポイントダウンロード@civitAI

In [ ]:
%%writefile model_ids.txt
501240
128713
11745
177164
178711
44398
81116
101605

In [ ]:
# @title ダウンロードＵＩ@civitAI
!apt install -y aria2
!git clone --depth=1 https://github.com/forester3/ImageGenSupporter.git

import sys
sys.path.append("./ImageGenSupporter")

from ImageGenSupporter import model_download_list_ids

model_download_list_ids.create_download_ui("model_ids.txt", "/content/ComfyUI/models/checkpoints")


### ダウンロードＵＩ@Huggingface

In [ ]:
%%writefile repo_list.json
{
  "repositories": [
    {
      "repo_id": "nuigurumi/basil_mix",
      "desc": "BasilMixモデル"
    },
    {
      "repo_id": "nuigurumi/basil_mix",
      "desc": "BasilMixモデル"
    }
  ]
}


In [ ]:
# @title ダウンロードＵＩ@HuggingFace
import json
import os
import shutil
from huggingface_hub import HfApi, hf_hub_download
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

display(HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">'))

api = HfApi()

json_path = "repo_list.json"

with open(json_path, "r") as f:
    data = json.load(f)

repositories = data["repositories"]

repo_files_map = {}
for repo in repositories:
    repo_id = repo["repo_id"]
    desc = repo.get("desc", "")
    try:
        files = api.list_repo_files(repo_id)
        model_files = [f for f in files if f.endswith((".ckpt", ".safetensors"))]
        repo_files_map[repo_id] = {
            "desc": desc,
            "files": model_files
        }
    except Exception as e:
        repo_files_map[repo_id] = {
            "desc": desc,
            "files": [],
            "error": str(e)
        }

options = []
for repo_id, info in repo_files_map.items():
    for filename in info["files"]:
        label = f"{repo_id} - {filename} ({info['desc']})"
        options.append((label, (repo_id, filename)))

if not options:
    print("モデルファイルがありません。")
else:
    select = widgets.SelectMultiple(
        options=options,
        description="モデル選択",
        rows=10,
        layout=widgets.Layout(width='50%')
    )
    button = widgets.Button(description="ダウンロード",  icon='download')
    output = widgets.Output()

    def get_hf_cache_dir():
        hf_home = os.getenv("HF_HOME")
        if hf_home:
            return hf_home
        home = os.path.expanduser("~")
        return os.path.join(home, ".cache", "huggingface")

    def download_selected_files(b):
        with output:
            clear_output()
            save_dir = "/content/ComfyUI/models/checkpoints"
            os.makedirs(save_dir, exist_ok=True)
            cache_dir = get_hf_cache_dir()
            for repo_id, filename in select.value:
                target_path = os.path.join(save_dir, filename)
                if os.path.exists(target_path):
                    print(f"{target_path} は既に存在します。スキップします。")
                    continue
                print(f"{filename} を {repo_id} からダウンロード中...")
                try:
                    downloaded_path = hf_hub_download(repo_id=repo_id, filename=filename, force_download=True)
                    shutil.copy(downloaded_path, target_path)
                    print(f"{filename} のダウンロード完了。")

                    cache_subdir = repo_id.replace('/', '--')
                    target_cache_subdir = os.path.join(cache_dir, cache_subdir)
                    if os.path.exists(target_cache_subdir):
                        shutil.rmtree(target_cache_subdir, ignore_errors=True)
                        print(f"キャッシュの {target_cache_subdir} を削除しました。")

                except Exception as e:
                    print(f"ダウンロード失敗: {filename} - {e}")

    button.on_click(download_selected_files)
    display(select, button, output)


### マニュアルモデルダウンロード -> /content/ComfyUI/models/checkpoints

In [ ]:
%%writefile paths_and_urls.txt
/content/ComfyUI/models/checkpoints
https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors

In [ ]:
# @title URL取得@ブラウザによるモデルダウンロード
import importlib
import sys
sys.path.append('./ImageGenSupporter')
from ImageGenSupporter import manual_url_download

manual_url_download.download("paths_and_urls.txt")

In [ ]:
# @title MEGAからファイルを同期(Colab連続時は不要)
import os
import subprocess
from pathlib import Path
from getpass import getpass

def run_cmd(cmd):
    print(f"$ {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("[stderr]", result.stderr)
    result.check_returncode()

# 1. ログイン情報
MEGA_EMAIL = input("MEGA email: ")
MEGA_PASSWORD = getpass("MEGA password: ")

if not MEGA_EMAIL or not MEGA_PASSWORD:
    raise RuntimeError("環境変数 MEGA_EMAIL または MEGA_PASSWORD が設定されていません")

# 2. MEGA にログイン（subprocess 経由）
print("🟡 MEGA にログイン中...")
process = subprocess.run(
    ["mega-login", MEGA_EMAIL, MEGA_PASSWORD],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
if process.returncode == 0:
    print("🟢 MEGA ログイン成功")
else:
    print("❌ MEGA ログイン失敗")
    print(process.stderr)

# 3. ComfyUI/output から最新の .png ファイルを取得
output_path = Path("ComfyUI/output")
output_path.mkdir(parents=True, exist_ok=True)

print("ファイル一覧取得中（ComfyUI/output）")
result = subprocess.run("mega-ls /ComfyUI/output", shell=True, text=True, capture_output=True)
files = result.stdout.strip().split("\n") if result.stdout else []

png_files = [f for f in files if f.endswith(".png")]
if png_files:
    latest_png = sorted(png_files)[-1]  # 名前順で最新（タイムスタンプ取得は MEGAcmd では困難）
    print(f"最新ファイル: {latest_png}")
    run_cmd(f'mega-get "/ComfyUI/output/{latest_png}" "{output_path}"')
else:
    print("ComfyUI/output に .png ファイルが見つかりませんでした")

# 4. ComfyUI/user/default/workflows から .json をすべて取得
workflow_path = Path("ComfyUI/user/default/workflows")
workflow_path.mkdir(parents=True, exist_ok=True)

print("ファイル一覧取得中（ComfyUI/user/default/workflows）")
result = subprocess.run("mega-ls /ComfyUI/user/default/workflows", shell=True, text=True, capture_output=True)
files = result.stdout.strip().split("\n") if result.stdout else []

json_files = [f for f in files if f.endswith(".json")]
if json_files:
    for jf in json_files:
        print(f"ダウンロード中: {jf}")
        run_cmd(f'mega-get "/ComfyUI/user/default/workflows/{jf}" "{workflow_path}"')
else:
    print("ComfyUI/user/default/workflows に .json ファイルが見つかりませんでした")

# 5. ログアウト処理（安全のため明示）
run_cmd("mega-logout")
print("✅MEGAからの同期終了")

# 保存ツール他

In [ ]:
# @title BackUpツールインストール(GPU不要)
!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb
!apt update && apt install -y ./megacmd-xUbuntu_22.04_amd64.deb

!git clone --depth=1 https://github.com/forester3/ImageGenSupporter.git

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

## user/default
!mkdir -p /content/ComfyUI/user
!ln -s /content/drive/MyDrive/ComfyUI/user/default /content/ComfyUI/user/default

## output
!ln -s /content/drive/MyDrive/ComfyUI/output /content/ComfyUI/output

print("✅ Backup Tool インストール完了")

In [ ]:
# @title MEGAに画像とワークフローを保存
import os
import subprocess
from pathlib import Path
from getpass import getpass

def mega_login(email, password):
    result = subprocess.run(['mega-login', email, password], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        raise RuntimeError("❌ MEGAログイン失敗: " + result.stderr)

def mega_logout():
    result = subprocess.run(['mega-logout'], capture_output=True, text=True)
    print(result.stdout)

def mega_upload(local_path, remote_path):
    local_path = str(local_path)
    result = subprocess.run(['mega-put', local_path, remote_path], capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"アップロード失敗: {local_path} -> {remote_path}\n{result.stderr}")
    print(f"アップロード成功: {local_path} -> {remote_path}")

# MEGAアカウント情報
MEGA_EMAIL = input("MEGA email: ")
MEGA_PASSWORD = getpass("MEGA password: ")

if not MEGA_EMAIL or not MEGA_PASSWORD:
    raise RuntimeError("❌ 環境変数 MEGA_EMAIL または MEGA_PASSWORD が設定されていません")

# ログイン実行
mega_login(MEGA_EMAIL, MEGA_PASSWORD)

# ComfyUI/output フォルダの全pngをアップロード
output_dir = Path("ComfyUI/output")
png_files = sorted(output_dir.glob("*.png"), key=lambda p: p.stat().st_mtime, reverse=True)
if png_files:
    print(f"アップロード対象のPNGファイル数: {len(png_files)}")
    for png_file in png_files:
        try:
            print(f"アップロード中: {png_file}")
            mega_upload(png_file, "/ComfyUI/output/")
        except Exception as e:
            print(f"エラー: {e}")
else:
    print("アップロード対象のpngファイルが見つかりません")

# ComfyUI/user/default/workflows のjsonファイルをすべてアップロード
workflow_dir = Path("ComfyUI/user/default/workflows")
workflow_files = list(workflow_dir.glob("*.json"))
if workflow_files:
    for wf_file in workflow_files:
        try:
            print(f"アップロード中: {wf_file}")
            mega_upload(wf_file, "/ComfyUI/user/default/workflows/")
        except Exception as e:
            print(f"エラー: {e}")
else:
    print("アップロード対象のワークフローファイルが見つかりません")

mega_logout()
print("✅ MEGAアップロード終了")